# 分析权利的游戏网络 @sumnous

几个月前，数学家Andrew Beveridge和Jie Shan在Math Horizon杂志上发表了权利的游戏的网络，其中，他们分析了小说“冰雨的风暴”，火爆的“冰与火之歌”（权利的游戏电视剧的基础）的第三卷中的角色互动网络。在他们的论文中，他们详细介绍了如何通过使用文本分析和实体抽取，来发现文本中提到的角色，从而构建角色互动网络。然后，他们应用社交网络分析算法到该网络中，以找到该网络中最重要的角色，并且应用社区检测算法来找到角色集群。

分析和可视化是通过使用Gephi，这一流行的图形分析工具，来完成的。近期也有很多作者使用Neo4j来复制结果。我们也可以使用igraph工具包来分析一下权利的有效网络。

参考：

https://www.macalester.edu/~abeverid/thrones.html

https://bigdata-ny.github.io/2016/08/12/graph-of-thrones-neo4j-social-network-analysis/

http://www.lyonwj.com/2016/06/26/graph-of-thrones-neo4j-social-network-analysis/

In [1]:
import csv
edges = []
firstline = True
with open('stormofswords.csv', 'rb') as f:
    for row in csv.reader(f.read().splitlines()):
        if firstline == True:
            firstline = False
            continue
        u, v, weight = [i for i in row]
        edges.append((u, v, int(weight)))

## python-igraph库

安装：

pip install python-igraph

或

conda install -c marufr python-igraph=0.7.1.post6

python-igraph manual: http://igraph.org/python/doc/igraph.Graph-class.html

In [2]:
from igraph import Graph as IGraph

g = IGraph.TupleList(edges, directed=True, vertex_name_attr='name', edge_attrs=None, weights=True)
# print g

In [3]:
names = g.vs["name"]
weights = g.es["weight"]
g.is_weighted()

True

## 分析网络

In [4]:
# 角色数
g.vcount()

107

In [5]:
# 网络直径: 一个网络的直径（或者测地线）被定义为网络中的最长最短路径。
print g.diameter()
names = g.vs["name"]
print g.get_diameter()
[names[x] for x in g.get_diameter()]

7
[38, 19, 24, 5, 0, 2, 9, 86]


['Bronn', 'Gregor', 'Sandor', 'Robert', 'Aemon', 'Samwell', 'Mance', 'Dalla']

## 最短路径

In [6]:
print g.shortest_paths("Jon","Margaery")
print "---------------------"
print [names[x] for x in g.get_shortest_paths("Jon","Margaery")[0]]
print "---------------------"
paths = g.get_all_shortest_paths("Jon")
for p in paths:
    print [names[x] for x in p]

[[3]]
---------------------
['Jon', 'Robert', 'Renly', 'Margaery']
---------------------
['Jon', 'Aemon']
['Jon', 'Grenn']
['Jon', 'Samwell']
['Jon', 'Robert']
['Jon', 'Alliser']
['Jon', 'Mance']
['Jon']
['Jon', 'Robert', 'Thoros']
['Jon', 'Stannis', 'Balon']
['Jon', 'Stannis', 'Balon', 'Loras']
['Jon', 'Robert', 'Renly', 'Loras']
['Jon', 'Stannis', 'Renly', 'Loras']
['Jon', 'Robert', 'Barristan']
['Jon', 'Meera']
['Jon', 'Theon']
['Jon', 'Stannis']
['Jon', 'Stannis', 'Renly', 'Varys', 'Pycelle']
['Jon', 'Robert', 'Renly', 'Varys', 'Pycelle']
['Jon', 'Robert', 'Renly', 'Varys']
['Jon', 'Stannis', 'Renly', 'Varys']
['Jon', 'Craster']
['Jon', 'Craster', 'Karl']
['Jon', 'Melisandre', 'Davos']
['Jon', 'Stannis', 'Davos']
['Jon', 'Stannis', 'Davos', 'Cressen']
['Jon', 'Melisandre', 'Davos', 'Cressen']
['Jon', 'Stannis', 'Davos', 'Salladhor']
['Jon', 'Melisandre', 'Davos', 'Salladhor']
['Jon', 'Eddison']
['Jon', 'Gilly']
['Jon', 'Stannis', 'Renly']
['Jon', 'Robert', 'Renly']
['Jon', 'Janos']

## 中心性度量

In [7]:
# 度中心性(Degree Centrality)
# 度中心性仅是一个节点在网络中的连接数。在权利的游戏的图的上下文中，一个角色的度中心性是该角色交互的其他角色数。
print g.maxdegree()
print "---------------------"
for p in g.vs:
    if p.degree() > 15:
        print p["name"],p.degree()

36
---------------------
Jaime 24
Robert 18
Tyrion 36
Tywin 22
Arya 19
Cersei 20
Joffrey 18
Jon 26
Catelyn 18
Robb 25
Sansa 26


In [8]:
# 加权度中心性
for p in g.vs:
    weightedDegree = sum([x.degree() for x in p.neighbors()])
    if weightedDegree > 250:
        print p["name"],weightedDegree

Jaime 307
Robert 262
Tyrion 357
Tywin 251
Arya 279
Cersei 273
Joffrey 262
Robb 301
Sansa 325


In [9]:
# 邻居平均度
for p in zip(g.vs,g.knn()[0]):
    if p[1] > 20:
        print p[0]["name"],p[1]

Aerys 25.0
Balon 21.6666666667
Jeyne 21.5
Petyr 20.7142857143
Pycelle 21.25
Qyburn 24.0
Myrcella 21.25
Orell 26.0
Ellaria 21.5
Mace 21.6666666667
Ramsay 25.0
Chataya 20.5
Doran 36.0
Walton 24.0


In [10]:
# 介数中心性(Betweenness Centrality)
# 一个网络中的一个节点的中介中心性(Betweenness Centrality) 是，网络中所有的节点对之间通过该节点的最短路径数。
# 中介中心性是一项重要的指标，因为它可以用于识别网络中的“信息代理”，或者那些连接不同集群的节点。
btvs = []
for p in zip(g.vs, g.betweenness()):
    btvs.append({"name": p[0]["name"], "bt": p[1]})
# print pgvs
sorted(btvs, key=lambda k: k['bt'], reverse=True)[:10]

[{'bt': 332.9746031746032, 'name': 'Tyrion'},
 {'bt': 244.63571428571433, 'name': 'Samwell'},
 {'bt': 226.2047619047619, 'name': 'Stannis'},
 {'bt': 208.62301587301587, 'name': 'Robert'},
 {'bt': 138.66666666666666, 'name': 'Mance'},
 {'bt': 119.99563492063493, 'name': 'Jaime'},
 {'bt': 114.33333333333334, 'name': 'Sandor'},
 {'bt': 111.26666666666665, 'name': 'Jon'},
 {'bt': 90.65, 'name': 'Janos'},
 {'bt': 64.59761904761905, 'name': 'Aemon'}]

In [11]:
# 接近中心性(Closeness centrality)
# 接近中心性(Closeness centrality)是到网络中所有其他角色的平均距离的倒数。
# 具有高接近中心性的节点通常在图中的集群之间被高度连接，但在集群外部不一定是高度连接的。
ccvs = []
for p in zip(g.vs, g.closeness()):
    ccvs.append({"name": p[0]["name"], "cc": p[1]})
# print pgvs
sorted(ccvs, key=lambda k: k['cc'], reverse=True)[:10]

[{'cc': 0.5120772946859904, 'name': 'Tyrion'},
 {'cc': 0.5096153846153846, 'name': 'Sansa'},
 {'cc': 0.5, 'name': 'Robert'},
 {'cc': 0.48847926267281105, 'name': 'Robb'},
 {'cc': 0.48623853211009177, 'name': 'Arya'},
 {'cc': 0.4796380090497738, 'name': 'Jaime'},
 {'cc': 0.4796380090497738, 'name': 'Jon'},
 {'cc': 0.4796380090497738, 'name': 'Stannis'},
 {'cc': 0.4690265486725664, 'name': 'Tywin'},
 {'cc': 0.4608695652173913, 'name': 'Eddard'}]

## PageRank算法

PageRank是一种最初由Google用来对网页重要性进行排序的算法。

In [12]:
pg = g.pagerank()
# pg = g.pagerank(vertices=None, directed=True, damping=0.85, 
#                 weights=weights, arpack_options=None, 
#                 implementation='prpack', 
#                 niter=1000, eps=0.001)
pgvs = []
for p in zip(g.vs, pg):
    pgvs.append({"name": p[0]["name"], "pg": p[1]})
# print pgvs
sorted(pgvs, key=lambda k: k['pg'], reverse=True)[:10]

[{'name': 'Margaery', 'pg': 0.032841464406084854},
 {'name': 'Samwell', 'pg': 0.025444623421744195},
 {'name': 'Loras', 'pg': 0.024829863132020076},
 {'name': 'Roslin', 'pg': 0.02410662426608122},
 {'name': 'Qhorin', 'pg': 0.02001318236350523},
 {'name': 'Karl', 'pg': 0.019198607452237473},
 {'name': 'Drogo', 'pg': 0.018892299206707614},
 {'name': 'Grenn', 'pg': 0.01762837304517548},
 {'name': 'Pycelle', 'pg': 0.01732234562876074},
 {'name': 'Craster', 'pg': 0.017148610457964602}]

## 社区检测（Community Detection）

社区检测算法用以查找图中的集群。我们将使用igraph中实现的walktrap社区检测方法，来找到那些在社区之中频繁交互，但在社区之外不存在太多互动的角色的社区。

In [13]:
clusters = IGraph.community_walktrap(g, weights="weight").as_clustering()
# community_walktrap: Community detection algorithm of Latapy & Pons, based on random walks.
# Pascal Pons, Matthieu Latapy: Computing communities in large networks using random walks, 
# http://arxiv.org/abs/physics/0512106.
nodes = [{"name": node["name"]} for node in g.vs]
community = {}
for node in nodes:
    idx = g.vs.find(name=node["name"]).index
    node["community"] = clusters.membership[idx]
    if node["community"] not in community:
        community[node["community"]] = [node["name"]]
    else:
        community[node["community"]].append(node["name"])
for c,l in community.iteritems():
    print "Community ", c, ": ", l

Community  0 :  ['Aemon', 'Grenn', 'Samwell', 'Alliser', 'Mance', 'Jon', 'Craster', 'Karl', 'Eddison', 'Gilly', 'Janos', 'Bowen', 'Dalla', 'Orell', 'Qhorin', 'Rattleshirt', 'Styr', 'Val', 'Ygritte']
Community  1 :  ['Aerys', 'Jaime', 'Robert', 'Tyrion', 'Tywin', 'Amory', 'Oberyn', 'Cersei', 'Gregor', 'Joffrey', 'Balon', 'Loras', 'Brienne', 'Bronn', 'Podrick', 'Lysa', 'Petyr', 'Sansa', 'Elia', 'Ilyn', 'Meryn', 'Pycelle', 'Shae', 'Varys', 'Qyburn', 'Renly', 'Tommen', 'Kevan', 'Margaery', 'Myrcella', 'Jon Arryn', 'Olenna', 'Marillion', 'Robert Arryn', 'Ellaria', 'Mace', 'Chataya', 'Doran', 'Walton']
Community  2 :  ['Arya', 'Anguy', 'Beric', 'Gendry', 'Sandor', 'Thoros', 'Eddard']
Community  3 :  ['Bran', 'Rickon', 'Hodor', 'Jojen', 'Luwin', 'Meera', 'Nan', 'Theon']
Community  4 :  ['Brynden', 'Roose', 'Lothar', 'Walder', 'Catelyn', 'Edmure', 'Hoster', 'Jeyne', 'Robb', 'Roslin', 'Rickard', 'Ramsay']
Community  5 :  ['Belwas', 'Barristan', 'Illyrio', 'Daario', 'Drogo', 'Irri', 'Daenerys', 

# 聚类算法

In [14]:
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets.samples_generator import make_blobs

In [15]:
# Generate sample data  
np.random.seed(0)  
  
batch_size = 45  
centers = [[1, 1], [-1, -1], [1, -1]] #初始化三个中心  
n_clusters = len(centers)       #聚类的数目为3  
#产生30000组两维的数据，以上边三个点为中心，以（-10,10）为边界，数据集的标准差是0.7  
X, labels_true = make_blobs(n_samples=30000, centers=centers, cluster_std=0.7)  

In [16]:
# Compute clustering with Means  
  
k_means = KMeans(init='kmeans++', n_clusters=3, n_init=10)  
t0 = time.time() #当前时间  
k_means.fit(X)  
#使用K-Means 对 30000数据集训练算法的时间消耗  
t_batch = time.time() - t0  

In [17]:
# Compute clustering with MiniBatchKMeans  
  
mbk = MiniBatchKMeans(init='kmeans++', n_clusters=3, batch_size=batch_size,  
                      n_init=10, max_no_improvement=10, verbose=0)  
t0 = time.time()  
mbk.fit(X)  
#使用MiniBatchKMeans 对 30000数据集训练算法的时间消耗  
t_mini_batch = time.time() - t0  

In [18]:
# Plot result  
  
#创建一个绘图对象, 并设置对象的宽度和高度, 如果不创建直接调用plot, Matplotlib会直接创建一个绘图对象  
#当绘图对象中有多个轴的时候，可以通过工具栏中的Configure Subplots按钮， 
#交互式地调节轴之间的间距和轴与边框之间的距离。 
#如果希望在程序中调节的话，可以调用subplots_adjust函数， 
#它有left, right, bottom, top, wspace, hspace等几个关键字参数， 
#这些参数的值都是0到1之间的小数，它们是以绘图区域的宽高为1进行正规化之后的坐标或者长度。   
fig = plt.figure(figsize=(8, 3))  
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)  
colors = ['#4EACC5', '#FF9C34', '#4E9A06']  
  
# We want to have the same colors for the same cluster from the  
# MiniBatchKMeans and the KMeans algorithm. Let's pair the cluster centers per  
# closest one.  
k_means_cluster_centers = np.sort(k_means.cluster_centers_, axis=0)  
mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis=0)  
k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)  
mbk_means_labels = pairwise_distances_argmin(X, mbk_means_cluster_centers)  
order = pairwise_distances_argmin(k_means_cluster_centers,  
                                  mbk_means_cluster_centers)  
# KMeans  
ax = fig.add_subplot(1, 3, 1) #add_subplot  图像分给为 一行三列，第一块  
for k, col in zip(range(n_clusters), colors):  
    my_members = k_means_labels == k  
    cluster_center = k_means_cluster_centers[k]  
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',  
            markerfacecolor=col, marker='.')  
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,  
            markeredgecolor='k', markersize=6)  
ax.set_title('KMeans')  
ax.set_xticks(())  
ax.set_yticks(())  
plt.text(-3.5, 1.8,  'train time: %.2fs\ninertia: %f' % (  
    t_batch, k_means.inertia_))  
  
# MiniBatchKMeans  
ax = fig.add_subplot(1, 3, 2)#add_subplot  图像分给为 一行三列，第二块  
for k, col in zip(range(n_clusters), colors):  
    my_members = mbk_means_labels == order[k]  
    cluster_center = mbk_means_cluster_centers[order[k]]  
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',  
            markerfacecolor=col, marker='.')  
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,  
            markeredgecolor='k', markersize=6)  
ax.set_title('MiniBatchKMeans')  
ax.set_xticks(())  
ax.set_yticks(())  
plt.text(-3.5, 1.8, 'train time: %.2fs\ninertia: %f' %  
         (t_mini_batch, mbk.inertia_))  
  
# Initialise the different array to all False  
different = (mbk_means_labels == 4)  
ax = fig.add_subplot(1, 3, 3)#add_subplot  图像分给为 一行三列，第三块  
  
for k in range(n_clusters):  
    different += ((k_means_labels == k) != (mbk_means_labels == order[k]))  
  
identic = np.logical_not(different)  
ax.plot(X[identic, 0], X[identic, 1], 'w',  
        markerfacecolor='#bbbbbb', marker='.')  
ax.plot(X[different, 0], X[different, 1], 'w',  
        markerfacecolor='m', marker='.')  
ax.set_title('Difference')  
ax.set_xticks(())  
ax.set_yticks(())  
  
plt.show()